In [2]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.17.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [3]:
from mip import *

In [ ]:
dias=7
requerimientos = [31, 45, 40, 40, 48, 30, 25]
turnos=[[1,1,1,1,1,0,0], # 0
        [0,1,1,1,1,1,0], # 1
        [0,0,1,1,1,1,1], # 2
        [1,0,0,1,1,1,1], # 3
        [1,1,0,0,1,1,1], # 4
        [1,1,1,0,0,1,1], # 5
        [1,1,1,1,0,0,1]] # 6

LOS DIAS DONDE TRABAJAN ESTAN EN LA MATRIX DE TURNOS

In [ ]:
#crear estructura para guardar el modelo
m = Model("personal",solver_name=CBC)

In [ ]:
# KNAPSACK -> k = [m.add_var(var_type=BINARY) for i in I]
#construir las variables
x = [m.add_var(var_type=INTEGER) for i in range(7)]

In [ ]:
#construir el objetivo
m.objective = minimize(xsum(x[i] for i in range(7)))
#construir cada restricción
for j in range(7):
    m += xsum(turnos[i][j]*x[i] for i in range(7)) >= requerimientos[j]


In [ ]:
# resolver
m.optimize()

<OptimizationStatus.OPTIMAL: 0>


```
#construir el objetivo
m.objective = minimize(xsum(x[i] for i in range(7)))
#construir cada restricción
for j in range(7):
    m += xsum(turnos[i][j]*x[i] for i in range(7)) == requerimientos[j]


```


```
#construir el objetivo
m.objective = minimize(xsum(x[i] for i in range(7)))
#construir cada restricción
for j in range(7):
    m += xsum(turnos[i][j]*x[i] for i in range(7)) <= requerimientos[j]
```




In [ ]:
print("numero de trabajadores",int(m.objective_value))
for i in range(7):
  print("turno",i,"trabajadores",int(x[i].x))
for j in range(7):
  print("necesidades",requerimientos[j],end=" ")
  acc=0
  for i in range(7):
    if turnos[i][j]==1 and x[i].x>0:
      print("(",i,",",x[i].x,") ",end="")
      acc += x[i].x
  print("total: ",acc)

numero de trabajadores 53
turno 0 trabajadores 18
turno 1 trabajadores 9
turno 2 trabajadores 8
turno 3 trabajadores 0
turno 4 trabajadores 13
turno 5 trabajadores 0
turno 6 trabajadores 5
necesidades 31 ( 0 , 18.0 ) ( 4 , 13.0 ) ( 6 , 5.0 ) total:  36.0
necesidades 45 ( 0 , 18.0 ) ( 1 , 9.0 ) ( 4 , 13.0 ) ( 6 , 5.0 ) total:  45.0
necesidades 40 ( 0 , 18.0 ) ( 1 , 9.0 ) ( 2 , 8.0 ) ( 6 , 5.0 ) total:  40.0
necesidades 40 ( 0 , 18.0 ) ( 1 , 9.0 ) ( 2 , 8.0 ) ( 6 , 5.0 ) total:  40.0
necesidades 48 ( 0 , 18.0 ) ( 1 , 9.0 ) ( 2 , 8.0 ) ( 4 , 13.0 ) total:  48.0
necesidades 30 ( 1 , 9.0 ) ( 2 , 8.0 ) ( 4 , 13.0 ) total:  30.0
necesidades 25 ( 2 , 8.0 ) ( 4 , 13.0 ) ( 6 , 5.0 ) total:  26.0


optimization problem:
- the optimal solution requires 53 workers in total.
- the distribution of workers across the 7 shifts. For example, shift 0 has 18 workers, shift 1 has 9, and so on.


## HW:

Modifica el modelo de asignación de trabajadores de analítica prescriptiva analizado en clase para que considere sueldos diferentes en cada uno de los posibles turnos.

Concretamente, considere que el coste de un turno de lunes a viernes es igual a 1, mientras que el sueldo de un turno que incluye el sábado o el domingo es igual a 1,15, y el sueldo de un turno que incluye ambos el sábado y el domingo es igual a 1,3.


Option 1:


In [28]:
from mip import Model, xsum, minimize, INTEGER

# Definición de constantes
dias = 7
requerimientos = [31, 45, 40, 40, 48, 30, 25]

# Definición de la disponibilidad de turnos: binary matrix
turnos = [
    [1, 1, 1, 1, 1, 0, 0],  # Turno 0: Lunes a Viernes
    [0, 1, 1, 1, 1, 1, 0],  # Turno 1: Martes a Sábado
    [0, 0, 1, 1, 1, 1, 1],  # Turno 2: Miércoles a Domingo
    [1, 0, 0, 1, 1, 1, 1],  # Turno 3: Lunes, Jueves a Domingo
    [1, 1, 0, 0, 1, 1, 1],  # Turno 4: Lunes, Martes, Viernes a Domingo
    [1, 1, 1, 0, 0, 1, 1],  # Turno 5: Lunes a Miércoles, Sábado a Domingo
    [1, 1, 1, 1, 0, 0, 1]   # Turno 6: Lunes a Jueves, Domingo
]

# Definición de los costos por turno
base_cost = 1  # Costo base para turnos de lunes a viernes
extra_weekend_15 = base_cost * 1.15  # Costo para turnos que incluyen sábado o domingo (15% extra)
extra_weekend_30 = base_cost * 1.3   # Costo para turnos que incluyen sábado y domingo (30% extra)

# Horas por turno
hours_per_day = 9
hours_per_shift = [sum(turno) * hours_per_day for turno in turnos]

In [29]:
# Crear la estructura para guardar el modelo
m = Model("personal", solver_name=CBC)

In [30]:
# Construir las variables
x = [m.add_var(var_type=INTEGER) for i in range(7)]

In [31]:
# Construir el objetivo: minimizar el costo total
m.objective = minimize(xsum(
    (base_cost * x[i] if turnos[i].count(1) == 5 else
     extra_weekend_15 * x[i] if turnos[i].count(1) == 6 else
     extra_weekend_30 * x[i])
    for i in range(7)
))

# Construir cada restricción
for j in range(7):
    m += xsum(turnos[i][j] * x[i] for i in range(7)) >= requerimientos[j]

In [32]:
# Resolver el modelo
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [33]:
# Imprimir resultados
total_workers = sum(v.x for v in m.vars)
print(f"Número total de trabajadores: {int(total_workers)}")
print(f"Costo total: {m.objective_value:.2f}\n")

for i in range(7):
    print(f"Turno {i} trabajadores: {int(x[i].x)}, Horas: {hours_per_shift[i]}")

for j in range(7):
    print(f"Necesidades {requerimientos[j]}", end=" ")
    acc = 0
    for i in range(7):
        if turnos[i][j] == 1 and x[i].x > 0:
            print(f"({i}, {int(x[i].x)}) ", end="")
            acc += x[i].x
    print(f"Total: {acc}")

Número total de trabajadores: 53
Costo total: 53.00

Turno 0 trabajadores: 18, Horas: 45
Turno 1 trabajadores: 9, Horas: 45
Turno 2 trabajadores: 8, Horas: 45
Turno 3 trabajadores: 0, Horas: 45
Turno 4 trabajadores: 13, Horas: 45
Turno 5 trabajadores: 0, Horas: 45
Turno 6 trabajadores: 5, Horas: 45
Necesidades 31 (0, 18) (4, 13) (6, 5) Total: 36.0
Necesidades 45 (0, 18) (1, 9) (4, 13) (6, 5) Total: 45.0
Necesidades 40 (0, 18) (1, 9) (2, 8) (6, 5) Total: 40.0
Necesidades 40 (0, 18) (1, 9) (2, 8) (6, 5) Total: 40.0
Necesidades 48 (0, 18) (1, 9) (2, 8) (4, 13) Total: 48.0
Necesidades 30 (1, 9) (2, 8) (4, 13) Total: 30.0
Necesidades 25 (2, 8) (4, 13) (6, 5) Total: 26.0


Necesidades 31: 31 trabajores son necesitados. 18 trabajadores fueron asignados al turno 0, 13 trabajadores fueron asignados al turno 4, y  5 trabajadores fueron asignados al turno 6. 5 trabajadores extra.

If we consider extra costs:

In [22]:
from mip import Model, xsum, minimize, INTEGER

# Definición de constantes
dias = 7
requerimientos = [31, 45, 40, 40, 48, 30, 25]
turnos = [
    [1, 1, 1, 1, 1, 0, 0],  # Turno 0: Lunes a Viernes
    [0, 1, 1, 1, 1, 1, 0],  # Turno 1: Martes a Sábado
    [0, 0, 1, 1, 1, 1, 1],  # Turno 2: Miércoles a Domingo
    [1, 0, 0, 1, 1, 1, 1],  # Turno 3: Lunes, Jueves a Domingo
    [1, 1, 0, 0, 1, 1, 1],  # Turno 4: Lunes, Martes, Viernes a Domingo
    [1, 1, 1, 0, 0, 1, 1],  # Turno 5: Lunes a Miércoles, Sábado a Domingo
    [1, 1, 1, 1, 0, 0, 1]   # Turno 6: Lunes a Jueves, Domingo
]

# Costos por turno
base_cost = 1
extra_weekend_15 = base_cost * 1.15
extra_weekend_30 = base_cost * 1.3

# Costos adicionales
overtime_cost = 0.2  # Costo por hora extra por trabajador
training_costs = [0, 0.5, 0.75, 0.5, 0.5, 0.75, 1]  # Costo de capacitación por trabajador por turno

# Horas por turno
hours_per_day = 9
hours_per_shift = [sum(turno) * hours_per_day for turno in turnos]

In [23]:
# Crear el modelo
m = Model("personal", solver_name=CBC)

In [24]:
# Variables de decisión
x = [m.add_var(var_type=INTEGER) for i in range(7)]

In [25]:
# Función objetivo: minimizar el costo total
m.objective = minimize(xsum(
    (base_cost * x[i] if turnos[i].count(1) == 5 else
     extra_weekend_15 * x[i] if turnos[i].count(1) == 6 else
     extra_weekend_30 * x[i]) +
    (overtime_cost * (hours_per_shift[i] - 8 * turnos[i].count(1)) * x[i] if hours_per_shift[i] > 8 * turnos[i].count(1) else 0) +
    (training_costs[i] * x[i])
    for i in range(7)
))

# Restricciones de necesidades diarias
for j in range(7):
    m += xsum(turnos[i][j] * x[i] for i in range(7)) >= requerimientos[j]

In [26]:
# Resolver el modelo
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [27]:
# Imprimir resultados
total_workers = sum(v.x for v in m.vars)
print(f"Número total de trabajadores: {int(total_workers)}")
print(f"Costo total: {m.objective_value:.2f}\n")

for i in range(7):
    print(f"Turno {i} trabajadores: {int(x[i].x)}")

for j in range(7):
    print(f"Necesidades {requerimientos[j]}", end=" ")
    acc = 0
    for i in range(7):
        if turnos[i][j] == 1 and x[i].x > 0:
            print(f"({i}, {int(x[i].x)}) ", end="")
            acc += x[i].x
    print(f"Total: {acc}")

Número total de trabajadores: 53
Costo total: 127.00

Turno 0 trabajadores: 19
Turno 1 trabajadores: 9
Turno 2 trabajadores: 8
Turno 3 trabajadores: 0
Turno 4 trabajadores: 13
Turno 5 trabajadores: 0
Turno 6 trabajadores: 4
Necesidades 31 (0, 19) (4, 13) (6, 4) Total: 36.0
Necesidades 45 (0, 19) (1, 9) (4, 13) (6, 4) Total: 45.0
Necesidades 40 (0, 19) (1, 9) (2, 8) (6, 4) Total: 40.0
Necesidades 40 (0, 19) (1, 9) (2, 8) (6, 4) Total: 40.0
Necesidades 48 (0, 19) (1, 9) (2, 8) (4, 13) Total: 49.0
Necesidades 30 (1, 9) (2, 8) (4, 13) Total: 30.0
Necesidades 25 (2, 8) (4, 13) (6, 4) Total: 25.0
